In [7]:
import pandas as pd
import xmltodict

In [8]:
data_path = r"C:\Users\dmatr\Downloads\data\Архитектура HypEx.wxml"

In [9]:
type_mapping = {
    "rectangle": "global",
    "rounded rectangle": "temporary"
}

status_mapping = {
    "#38761d": "complete",
    "#0b5394": "in_progress",
    "#525c61": "planed"
}

In [28]:
def parse_node(tasks:list, node:dict, area: str, parent: dict = None):
    parent = parent or {}
    result = None
    try:
        result = {
            "task": node["@text"],
            "area": area,
            "type": type_mapping.get(node.get("@shape")) or parent.get("type"),
            "status": type_mapping.get(node.get("@bgColor")) or parent.get("status"),
            "worker": node.get('eicon', {}).get('@id')
        }
        tasks.append(result)
    except KeyError as e:
        print(e)
        print(f"In node :\n{node}")

    topic = node.get("topic", [])
    topic = topic if isinstance(topic, list) else [topic]
    for t in topic:
        parse_node(tasks, t, area, result)

In [29]:
with open(data_path, "r", encoding="utf-8") as f:
    xml_string = f.read()
xml_data = xmltodict.parse(xml_string)
xml_data

{'map': {'@name': '1683982',
  '@theme': 'dark-prism',
  '@version': 'tango',
  'topic': {'@central': 'true',
   '@text': 'Архитектура HypEx',
   '@id': '1',
   'topic': [{'@position': '154,-412',
     '@order': '0',
     '@text': 'Dataset',
     '@id': '2',
     'topic': [{'@position': '277,-447',
       '@order': '0',
       '@text': 'ExperimentData',
       '@shape': 'rectangle',
       '@id': '3',
       '@bgColor': '#38761d',
       '@brColor': '#000000',
       'topic': {'@position': '475,-447',
        '@order': '0',
        '@text': 'Функции управления и атрибуты',
        '@id': '4',
        'topic': [{'@position': '713,-448',
          '@order': '1',
          '@text': 'Обёртка groupby в Dataset',
          '@shape': 'rounded rectangle',
          '@id': '6',
          '@bgColor': '#0b5394',
          'eicon': {'@id': '🦊'}},
         {'@position': '681,-411',
          '@order': '2',
          '@text': 'Функции создания',
          '@shape': 'rounded rectangle',
          '@i

In [30]:
tasks = []
main_node = xml_data["map"]["topic"]["topic"]

for area in main_node:
    parse_node(tasks,area, area["@text"])
    

'type'
In node :
{'@position': '154,-412', '@order': '0', '@text': 'Dataset', '@id': '2', 'topic': [{'@position': '277,-447', '@order': '0', '@text': 'ExperimentData', '@shape': 'rectangle', '@id': '3', '@bgColor': '#38761d', '@brColor': '#000000', 'topic': {'@position': '475,-447', '@order': '0', '@text': 'Функции управления и атрибуты', '@id': '4', 'topic': [{'@position': '713,-448', '@order': '1', '@text': 'Обёртка groupby в Dataset', '@shape': 'rounded rectangle', '@id': '6', '@bgColor': '#0b5394', 'eicon': {'@id': '🦊'}}, {'@position': '681,-411', '@order': '2', '@text': 'Функции создания', '@shape': 'rounded rectangle', '@id': '7'}, {'@position': '652,-484', '@order': '0', '@text': 'update', '@shape': 'rounded rectangle', '@id': '5', '@bgColor': '#0b5394', 'eicon': {'@id': '🦊'}}]}}, {'@position': '236,-376', '@order': '1', '@text': 'ID', '@shape': 'rectangle', '@id': '8', '@bgColor': '#38761d', '@brColor': '#000000', 'topic': {'@position': '320,-376', '@order': '0', '@text': 'ID-N

In [31]:
tasks

[]